# <font color="purple">Adversarial Training of Residual Network</font>

Definition of WideResNet and Architecture:

Residual Networks vs Wide Residual Networks

##  <center>The Result of Residual Network </center>

***Road Map***
* Data Preprocessing
* Model Cross Validation Results
* Evaluate the GridSearchCV Results
* Model Training and Learning Curves
* Model Adversarial Training Approach.


**Libraries**

In [ ]:
import sys
sys.path.insert(1,'/home/sefika/AE_Parseval_Network/src/cleverhans/future/tf2/attacks')
import cleverhans


In [2]:
!pip install -qq -e git+http://github.com/tensorflow/cleverhans.git#egg=cleverhans
import sys
sys.path.append('/content/src/cleverhans')
import cleverhans

     |████████████████████████████████| 163kB 4.3MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 


In [3]:
import tensorflow as tf
from cleverhans.future.tf2.attacks import fast_gradient_method
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from wresnet import WideResidualNetwork
from sklearn.model_selection import  KFold
import gzip
import pickle
import numpy as np

import warnings
warnings.filterwarnings("ignore")

print("\nTensorflow Version: " + tf.__version__)


# defined utility functions
from preprocessing import preprocessing_data
from training import train
from _utility import lrate


Tensorflow Version: 2.3.0


## <font color="green"> Data Preprocessing </font>


* Read Data from File

In [4]:
def read_data():
    with open("data.pz", 'rb') as file_:
        with gzip.GzipFile(fileobj=file_) as gzf:
            data = pickle.load(gzf, encoding='latin1', fix_imports=True)
    return data
data = read_data()

* Call data preprocessing function

In [5]:
X, y = preprocessing_data(data)
X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size = 0.1)
x_train, X_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.1)

## <font color="green"> Utilize Functions </font>

* Flipping the image using data augmentation technique

In [6]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,)

### <font color = "green">Some Parameters Regarding Adversarial Examples</font>

In [7]:
# predefined epsilon values
epsilon_list = [0.003,0.005,0.01,0.02]

### <font>General Visulazation Methods</font>

* Show accurancy and loss function of the results

## <font color="purple"> 1.) Baseline of the Model</font>

* Explain the Baseline Model here

In [8]:
EPOCHS = 50
BS = 64
init = (32, 32,1)
sgd = SGD(lr=0.1, momentum=0.9)
parameter = {'epochs': EPOCHS, 'batch_size': BS, 'optimizer': sgd}

In [9]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
callbacks_list = [lrate, es]
wresnet_ins = WideResidualNetwork(init, 0.0001, 0.9, nb_classes=4, N=2, k=1, dropout=0.0)

In [10]:
result_df = train(wresnet_ins, X_train, Y_train, X_test, y_test, EPOCHS, BS, sgd, generator, callbacks_list, epsilon_list)

conv2:channel:  -1
Wide Residual Network-16-1 created.
Finished compiling
Epoch 1/50
72/72 [==============================] - 2s 31ms/step - loss: 1.4235 - acc: 0.3346 - val_loss: 1.3856 - val_acc: 0.3864
Epoch 2/50
72/72 [==============================] - 2s 22ms/step - loss: 1.3476 - acc: 0.4136 - val_loss: 1.3344 - val_acc: 0.4291
Epoch 3/50
72/72 [==============================] - 2s 23ms/step - loss: 1.2887 - acc: 0.4755 - val_loss: 1.3175 - val_acc: 0.4563
Epoch 4/50
72/72 [==============================] - 2s 23ms/step - loss: 1.2531 - acc: 0.4947 - val_loss: 1.2215 - val_acc: 0.4893
Epoch 5/50
72/72 [==============================] - 2s 23ms/step - loss: 1.2081 - acc: 0.5306 - val_loss: 1.1695 - val_acc: 0.5301
Epoch 6/50
72/72 [==============================] - 2s 23ms/step - loss: 1.1719 - acc: 0.5414 - val_loss: 1.1019 - val_acc: 0.5689
Epoch 7/50
72/72 [==============================] - 2s 23ms/step - loss: 1.1177 - acc: 0.5667 - val_loss: 1.0589 - val_acc: 0.5981
Epoch 8/5

In [11]:
result_df["clean_mean"] = np.sum(result_df['acc_clean'])/10.0
result_df["0.003_mean"] = np.sum(result_df['0.003_acc'])/10.0
result_df["0.005_mean"] = np.sum(result_df['0.005_acc'])/10.0
result_df["0.02_mean"] = np.sum(result_df['0.02_acc'])/10.0
result_df["0.01_mean"] = np.sum(result_df['0.01_acc'])/10.0

In [13]:
column = ["clean_mean","0.003_mean","0.005_mean","0.02_mean","0.01_mean"]
result_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.724782,0.676265,0.641536,0.558115,0.41274


## <font color="purple">2.) Adversarial Training on Baseline Model</font>

In [ ]:
from adversarial_training import  AdversarialTraining
adversarial_training =  AdversarialTraining(parameter)
result_adv_df = adversarial_training.train(wresnet_ins, X_train, Y_train, X_test, y_test, epsilon_list, callbacks_list)

### <font color="purple"> Results of Adversarial Training</font>

In [16]:
result_adv_df["clean_mean"] = np.sum(result_adv_df['acc_clean'])/10.0
result_adv_df["0.003_mean"] = np.sum(result_adv_df['0.003_acc'])/10.0
result_adv_df["0.005_mean"] = np.sum(result_adv_df['0.005_acc'])/10.0
result_adv_df["0.02_mean"] = np.sum(result_adv_df['0.02_acc'])/10.0
result_adv_df["0.01_mean"] = np.sum(result_adv_df['0.01_acc'])/10.0

In [17]:
result_adv_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.718848,0.666492,0.633682,0.557068,0.416928


# <font color="blue">Compare Non-Adversarial Training with Adversarial Training</font>

In [18]:
result_adv_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.718848,0.666492,0.633682,0.557068,0.416928


In [19]:
result_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.724782,0.676265,0.641536,0.558115,0.41274


# <font color="blue">Conclusion</font>